# CatBoost

## Importing the libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost

In [ ]:
test_path = '/content/drive/My Drive/Datasets/Hacker_Earth_Challenges/Adopt_a_pet/Dataset/test.csv'
train_path = '/content/drive/My Drive/Datasets/Hacker_Earth_Challenges/Adopt_a_pet/Dataset/train.csv'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [ ]:
df = pd.read_csv(train_path)
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan, strategy="mean" )
df["condition"]=imp.fit_transform(df[["condition"]]).ravel()
X = df.iloc[:, 3:9].values
y = df.iloc[:, 9:11].values

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
y_train1 = y_train[:,0]
y_train2 = y_train[:,1]
y_test1 = y_test[:,0]
y_test2 = y_test[:, 1]


In [ ]:

# X_train[0]
print(X_train.shape,
y_train.shape)

(15067, 6) (15067, 2)


## Training CatBoost on the Training set

# for 1st classifier

best_params = {
  learning_rate =[0.357999,0.148553,0.130635],
  iterations = [51, 78, 90],
  acc = [90.92, 90.84, 90.73]
}

In [ ]:
from catboost import CatBoostClassifier #learning_rate = 0.148553,
classifier1 = CatBoostClassifier(iterations = 55,learning_rate =0.357999,cat_features =[1],loss_function='MultiClass') #learning_rate=0.05,
classifier1.fit(X_train, y_train1, eval_set=(X_test,y_test1))

In [ ]:
# for 1st target value
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier1.predict(X_test)
cm = confusion_matrix(y_test1, y_pred)
print(cm)
score = accuracy_score(y_test1, y_pred)*100
print(score)

[[1653  124    0]
 [ 218 1458    0]
 [   0    0  314]]
90.92115741969738


# for 2nd classifier

best_params = {
  learning_rate =[0.240278, 0.113996, 0.435139],
  iterations = [58, 137, 28],
  acc = [88.00, 87.64, 87.60]
}

In [ ]:
# learning_rate = 0.106326,iterations = 176
classifier2 = CatBoostClassifier(iterations =176,learning_rate = 0.106326,cat_features =[1],loss_function='MultiClass', use_best_model=True) #learning_rate=0.05,
# --per-float-feature-quantization 0:border_count=1024;1:border_count=1024
classifier2.fit(X_train, y_train2, eval_set=(X_test,y_test2), plot=True)

In [ ]:
y_pred2 = classifier2.predict(X_test)
#for 2nd target value
cm2 = confusion_matrix(y_test2, y_pred2)
print(cm2)
score2 = accuracy_score(y_test2, y_pred2)*100
print(score2)

[[   2    3    4    4]
 [   0 1148  304    7]
 [   0  107 2003    4]
 [   0   14   18  149]]
87.65595964958854


iterations = 142 , acc = 87.7621449429254


bestTest = 0.1444441723
bestIteration = 494

for iterations = 85 , acc = 91.

bestTest = 0.1449934695
bestIteration = 41 , acc = 90.5

## f1_score

In [ ]:
from sklearn.metrics import f1_score
score = f1_score(y_test1, y_pred, average='weighted') * 100
score2 = f1_score(y_test2, y_pred2, average='weighted') * 100
print(score, score2)

90.90715551745532 87.71086087590952


## Applying k-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies1 = cross_val_score(estimator = classifier1, X = X_train, y = y_train1, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies1.mean()*100))
accuracies = cross_val_score(estimator = classifier2, X = X_train, y = y_train2, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))

# Grid Search

Best Accuracy: 89.87 %
Best Parameters: {'learning_rate': 0.03} for 1st target

Best Accuracy: 87.77 %
Best Parameters: {'learning_rate': 0.03} for 2nd target

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = {'learning_rate': [] #0.01, 0.03, 0.1, 0.5, 1  = 0.03 87.62
}
grid_search = GridSearchCV(estimator = classifier2,
                           param_grid = grid,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1
                          )
grid_search = grid_search.fit(X_train, y_train2)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

#Preprocess the test set for submission

In [ ]:
test_Set = pd.read_csv(test_path)
print(test_Set.info())
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan, strategy="mean" )
test_Set["condition"]=imp.fit_transform(test_Set[["condition"]]).ravel()
X_test_final = test_Set.iloc[:, 3:9].values
X_test_final[0]

In [ ]:
len(set(test_Set['color_type']))

In [ ]:
y_pred_breedcategory = classifier1.predict(X_test_final)
predicted_breed_category = [int(i) for i in y_pred_breedcategory]

y_pred_petcategory = classifier2.predict(X_test_final)
predicted_pet_category = [int(i) for i in y_pred_petcategory]

In [ ]:
d = {'pet_id':test_Set['pet_id'], 'breed_category':predicted_breed_category, 'pet_category':predicted_pet_category}

In [ ]:
datafra = pd.DataFrame(d)
# d.to_csv('Submission.csv', index=False)
datafra

,pet_id,breed_category,pet_category
0,ANSL_75005,1,2
1,ANSL_76663,0,1
2,ANSL_58259,0,2
3,ANSL_67171,0,1
4,ANSL_72871,0,2
...,...,...,...
8067,ANSL_66809,0,2
8068,ANSL_59041,1,2
8069,ANSL_60034,1,2
8070,ANSL_58066,2,4


In [ ]:
datafra.to_csv('Submission5.csv',index=False) 
# got 80 marks subbimision1.csv  
# got 60 for subimision2.csv 
# got 89.21 for subimission3.csv
# got 89.45 for subimission4.csv
